In [1]:
"""
Compare Two Models on GlobalOpinionsQA

========================================
SIMPLIFIED WORKFLOW - JUST SET 2 COUNTRIES!
========================================

⭐ QUICK START:
   1. Go to CELL 1b (line ~50)
   2. Change these two lines:
      COUNTRY_A = Country.MEXICO  # Change to: US, UK, CHILE, or MEXICO
      COUNTRY_B = Country.CHILE   # Change to: US, UK, CHILE, or MEXICO
   3. Done! Everything else is automatic.

Metrics Computed:
    • JS Similarity: Overall distributional alignment (1 - Jensen-Shannon Distance)
    • Agreement Rate: How often model's argmax matches human majority choice

Usage:

    LOCAL:
        1. Set COUNTRY_A and COUNTRY_B in CELL 1b (options: US, UK, CHILE, MEXICO)
        2. Extract model zips to ./models/<country>/
        3. Run: python scripts/evaluate_two_models.py

    GOOGLE COLAB:
        1. Upload this file to Colab
        2. Edit CELL 1b: Set COUNTRY_A and COUNTRY_B (e.g., Country.MEXICO, Country.CHILE)
        3. Run CELL 1 (imports + config)
        4. Run CELL 2 and upload your two model zip files
           - Expected: <country>_dpo_model.zip (e.g., mexico_dpo_model.zip)
           - Created in training with: shutil.make_archive("mexico_dpo_model", 'zip', "results/dpo_models/mexico/final")
        5. Run remaining cells in order - everything auto-configured!

    The script automatically:
        ✓ Sets model paths based on country names
        ✓ Sets evaluation countries (evaluates on both countries' ground truth)
        ✓ Names models appropriately
        ✓ Saves results to <country_a>_vs_<country_b>_comparison.json

    You only need to change ONE thing: COUNTRY_A and COUNTRY_B in CELL 1b!
"""

# ============================================================================
# CELL 1: Imports and Setup
# ============================================================================
import torch
import numpy as np
import json
from pathlib import Path
from tqdm import tqdm
import ast
from scipy.spatial.distance import jensenshannon
from scipy.special import softmax
from scipy import stats
from enum import Enum

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from datasets import load_dataset

print("="*80)
print("TWO-MODEL COMPARISON ON GLOBALOPINIONSQA")
print("Metrics: JS Similarity & Agreement Rate")
print("="*80)


# ============================================================================
# CELL 1b: Country Configuration (EDIT THIS!)
# ============================================================================
class Country(Enum):
    """Valid countries for model comparison."""
    US = "us"
    UK = "uk"
    CHILE = "chile"
    MEXICO = "mexico"

    @property
    def display_name(self):
        """Get display name for the country."""
        return {
            Country.US: "United States",
            Country.UK: "United Kingdom",
            Country.CHILE: "Chile",
            Country.MEXICO: "Mexico"
        }[self]

    @property
    def globalopinions_names(self):
        """Get country names as they appear in GlobalOpinionsQA dataset."""
        return {
            Country.US: ["United States"],
            Country.UK: ["Britain", "Great Britain"],  # UK appears as Britain in dataset
            Country.CHILE: ["Chile"],
            Country.MEXICO: ["Mexico"]
        }[self]

    @property
    def short_name(self):
        """Get short display name for model naming."""
        return {
            Country.US: "US",
            Country.UK: "UK",
            Country.CHILE: "Chile",
            Country.MEXICO: "Mexico"
        }[self]


# ============================================================================
# ⭐ CONFIGURE YOUR COUNTRIES HERE ⭐
# ============================================================================
# Set which two countries' models you want to compare
# Options: Country.US, Country.UK, Country.CHILE, Country.MEXICO

COUNTRY_A = Country.UK  # First model
COUNTRY_B = Country.MEXICO   # Second model

# ============================================================================
# Everything below is automatic - no need to change anything!
# ============================================================================

print(f"\n✓ Configuration:")
print(f"  Model A: {COUNTRY_A.short_name} ({COUNTRY_A.value})")
print(f"  Model B: {COUNTRY_B.short_name} ({COUNTRY_B.value})")
print(f"  Will evaluate on: {COUNTRY_A.display_name} and {COUNTRY_B.display_name}")
print(f"\n  Expected zip files:")
print(f"    - {COUNTRY_A.value}_dpo_model.zip")
print(f"    - {COUNTRY_B.value}_dpo_model.zip")

TWO-MODEL COMPARISON ON GLOBALOPINIONSQA
Metrics: JS Similarity & Agreement Rate

✓ Configuration:
  Model A: UK (uk)
  Model B: Mexico (mexico)
  Will evaluate on: United Kingdom and Mexico

  Expected zip files:
    - uk_dpo_model.zip
    - mexico_dpo_model.zip


In [2]:
# # ============================================================================
# # CELL 2: Upload and Unzip Model Checkpoints (if running in Colab)
# # No need to re-run if the checkpoint is already stored in colab's files. might get errors otherwise
# # ============================================================================
# import zipfile
# import shutil
# import os

# def upload_and_extract_models(country_a, country_b):
#     """
#     Upload and extract model checkpoints from zip files.
#     Automatically configured based on COUNTRY_A and COUNTRY_B.
#     """
#     print(f"\n{'='*80}")
#     print("UPLOAD MODEL CHECKPOINTS")
#     print(f"{'='*80}")
#     print(f"\nPlease upload the following zip files:")
#     print(f"  1. {country_a.value}_dpo_model.zip (Model A: {country_a.short_name})")
#     print(f"  2. {country_b.value}_dpo_model.zip (Model B: {country_b.short_name})")
#     print("\n(If not running in Colab, skip this cell and manually extract to ./models/)")

#     try:
#         from google.colab import files

#         print(f"\n📤 Upload Model A ({country_a.short_name}) zip file:")
#         print(f"   Expected filename: {country_a.value}_dpo_model.zip")
#         uploaded_a = files.upload()

#         print(f"\n📤 Upload Model B ({country_b.short_name}) zip file:")
#         print(f"   Expected filename: {country_b.value}_dpo_model.zip")
#         uploaded_b = files.upload()

#         # Validate uploaded files
#         expected_files = {
#             country_a.value: f"{country_a.value}_dpo_model.zip",
#             country_b.value: f"{country_b.value}_dpo_model.zip"
#         }

#         uploaded_files = list(uploaded_a.keys()) + list(uploaded_b.keys())

#         # Extract both
#         models_dir = Path("./models")
#         models_dir.mkdir(exist_ok=True)

#         extracted_paths = {}

#         for zip_file in uploaded_files:
#             # Extract model name from zip filename
#             model_name = zip_file.replace("_dpo_model.zip", "").replace(".zip", "")
#             extract_path = models_dir / model_name
#             extract_path.mkdir(exist_ok=True, parents=True)

#             print(f"\n📦 Extracting {zip_file} to {extract_path}...")
#             with zipfile.ZipFile(zip_file, 'r') as zip_ref:
#                 zip_ref.extractall(extract_path)

#             extracted_paths[model_name] = str(extract_path)
#             print(f"✓ Extracted to {extract_path}")

#         # Verify both expected models were uploaded
#         if country_a.value not in extracted_paths:
#             print(f"\n⚠️  Warning: Expected {country_a.value}_dpo_model.zip but didn't find it")
#         if country_b.value not in extracted_paths:
#             print(f"\n⚠️  Warning: Expected {country_b.value}_dpo_model.zip but didn't find it")

#         print(f"\n✅ All models extracted!")
#         print(f"Model paths:")
#         for name, path in extracted_paths.items():
#             print(f"  {name}: {path}")

#         return extracted_paths

#     except ImportError:
#         print("⚠️ Not running in Colab. Please manually extract zip files to ./models/")
#         print("Example:")
#         print(f"  unzip {country_a.value}_dpo_model.zip -d ./models/{country_a.value}/")
#         print(f"  unzip {country_b.value}_dpo_model.zip -d ./models/{country_b.value}/")
#         return None

# extracted_paths = upload_and_extract_models(COUNTRY_A, COUNTRY_B)

In [3]:
# ============================================================================
# CELL 3: Auto-Generated Configuration
# ============================================================================
# All settings are automatically derived from COUNTRY_A and COUNTRY_B
# No manual configuration needed!

class Config:
    """Evaluation configuration - automatically generated from country selection."""

    def __init__(self, country_a: Country, country_b: Country):
        # Model paths (automatically set to extracted locations)
        self.model_a_path = f"./models/{country_a.value}"
        self.model_b_path = f"./models/{country_b.value}"

        # Model names for display
        self.model_a_name = f"f_A ({country_a.short_name})"
        self.model_b_name = f"f_B ({country_b.short_name})"

        # Countries being compared
        self.country_a = country_a
        self.country_b = country_b

        # Target countries for evaluation (evaluate on both countries' ground truth)
        # Use all variant names from GlobalOpinionsQA (needed for dataset filtering)
        self.target_countries = (
            country_a.globalopinions_names +
            country_b.globalopinions_names
        )

        # Remove duplicates while preserving order
        seen = set()
        self.target_countries = [
            x for x in self.target_countries
            if not (x in seen or seen.add(x))
        ]

        # Canonical country names for display (UK variants -> "United Kingdom")
        self.canonical_countries = []
        seen_canonical = set()
        for country_name in self.target_countries:
            if country_name.lower() in ['britain', 'great britain', 'uk', 'united kingdom']:
                canonical = "United Kingdom"
            else:
                canonical = country_name

            if canonical not in seen_canonical:
                self.canonical_countries.append(canonical)
                seen_canonical.add(canonical)

        # Base model (used for loading LoRA adapters)
        self.base_model = "Qwen/Qwen2.5-0.5B"

        # Evaluation settings
        self.max_samples = None  # Set to a number (e.g., 500) for quick testing

        # Output file
        self.output_file = f"./results/{country_a.value}_vs_{country_b.value}_comparison.json"

# Create configuration from selected countries
config = Config(COUNTRY_A, COUNTRY_B)

print(f"\n{'='*80}")
print("AUTO-GENERATED CONFIGURATION")
print(f"{'='*80}")
print(f"\nModel paths:")
print(f"  Model A: {config.model_a_path}")
print(f"  Model B: {config.model_b_path}")
print(f"\nModel names:")
print(f"  Model A: {config.model_a_name}")
print(f"  Model B: {config.model_b_name}")
print(f"\nEvaluation countries:")
for country in config.target_countries:
    print(f"  - {country}")
print(f"\nOutput file: {config.output_file}")
print(f"Max samples: {config.max_samples if config.max_samples else 'All (full dataset)'}")

print(f"\nConfiguration:")
print(f"  Model A: {config.model_a_name} at {config.model_a_path}")
print(f"  Model B: {config.model_b_name} at {config.model_b_path}")
print(f"  Base model: {config.base_model}")
print(f"  Target countries: {config.target_countries}")
print(f"  Max samples: {config.max_samples if config.max_samples else 'All'}")
print(f"  Output: {config.output_file}")


AUTO-GENERATED CONFIGURATION

Model paths:
  Model A: ./models/uk
  Model B: ./models/mexico

Model names:
  Model A: f_A (UK)
  Model B: f_B (Mexico)

Evaluation countries:
  - Britain
  - Great Britain
  - Mexico

Output file: ./results/uk_vs_mexico_comparison.json
Max samples: All (full dataset)

Configuration:
  Model A: f_A (UK) at ./models/uk
  Model B: f_B (Mexico) at ./models/mexico
  Base model: Qwen/Qwen2.5-0.5B
  Target countries: ['Britain', 'Great Britain', 'Mexico']
  Max samples: All
  Output: ./results/uk_vs_mexico_comparison.json


In [4]:
# ============================================================================
# CELL 4: Model Loading Functions
# ============================================================================
def load_model(model_path, base_model, model_name):
    """Load a trained model (with or without LoRA adapters)."""
    print(f"\nLoading {model_name}...")
    print(f"  Path: {model_path}")

    model_path = Path(model_path)

    if not model_path.exists():
        raise FileNotFoundError(f"Model not found: {model_path}")

    # Check GPU availability
    if not torch.cuda.is_available():
        raise RuntimeError("CUDA is not available! This script requires a GPU.")

    print(f"  Loading base model on GPU...")
    # Load base model - force to GPU with device_map
    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        device_map="cuda:0",  # Explicitly specify cuda:0 instead of "auto"
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
    )

    # Load LoRA adapters if present
    if (model_path / "adapter_config.json").exists():
        print(f"  Loading LoRA adapters...")
        model = PeftModel.from_pretrained(model, str(model_path))
        print(f"  Merging LoRA with base model...")
        model = model.merge_and_unload()

        # Explicitly ensure model is on GPU and in correct dtype after merge
        print(f"  Moving merged model to GPU...")
        model = model.to(device="cuda:0", dtype=torch.bfloat16)

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model.eval()

    # Verify model is on GPU
    device = next(model.parameters()).device
    dtype = next(model.parameters()).dtype
    print(f"✓ {model_name} loaded")
    print(f"  Device: {device} {'✓' if device.type == 'cuda' else '❌ WARNING: NOT ON GPU!'}")
    print(f"  Dtype: {dtype}")

    if device.type != 'cuda':
        raise RuntimeError(f"Model ended up on {device}, not GPU! This will be extremely slow.")

    return model, tokenizer

In [5]:
# ============================================================================
# CELL 5: Helper Functions
# ============================================================================
def parse_selections(selections_str):
    """Parse the selections string from GlobalOpinionsQA."""
    if 'defaultdict' in selections_str:
        dict_start = selections_str.find('{')
        dict_end = selections_str.rfind('}') + 1
        dict_str = selections_str[dict_start:dict_end]
        return ast.literal_eval(dict_str)
    return ast.literal_eval(selections_str)


def parse_options(options_str):
    """Parse options string into a list."""
    if isinstance(options_str, list):
        return options_str
    return ast.literal_eval(options_str)

In [6]:
# ============================================================================
# CELL 6: JS Similarity Evaluation Functions
# ============================================================================
def get_model_probabilities(model, tokenizer, question, options):
    """Get model's probability distribution over answer options."""
    if isinstance(options, str):
        options = parse_options(options)

    prompt = f"""Answer the following survey question by selecting one of the given options.

Question: {question}

Options:
"""
    for i, option in enumerate(options):
        prompt += f"{i+1}. {option}\n"

    prompt += "\nYour answer (just the number): "

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model(**inputs)
        next_token_logits = outputs.logits[0, -1, :]

    # Get logits for option numbers
    option_tokens = [tokenizer.encode(str(i+1), add_special_tokens=False)[0]
                     for i in range(len(options))]
    option_logits = next_token_logits[option_tokens]

    probs = softmax(option_logits.cpu().float().numpy())
    return probs.tolist(), len(options)


def compute_js_similarity(p, q):
    """Compute 1 - Jensen-Shannon Distance."""
    p = np.array(p)
    q = np.array(q)

    # Normalize
    p = p / np.sum(p) if np.sum(p) > 0 else p
    q = q / np.sum(q) if np.sum(q) > 0 else q

    # Add epsilon
    epsilon = 1e-10
    p = p + epsilon
    q = q + epsilon
    p = p / np.sum(p)
    q = q / np.sum(q)

    return 1 - jensenshannon(p, q)


def evaluate_js_similarity(model, tokenizer, model_name, dataset_items, target_countries):
    """Evaluate model using JS similarity metric."""
    print(f"\nEvaluating {model_name} with JS similarity...")

    # Create canonical country mapping (e.g., "Britain" and "Great Britain" -> "United Kingdom")
    canonical_map = {}
    canonical_names = set()

    for country_name in target_countries:
        # Check if this is a UK variant
        if country_name.lower() in ['britain', 'great britain', 'uk', 'united kingdom']:
            canonical = "United Kingdom"
            canonical_map[country_name] = canonical
            canonical_names.add(canonical)
        else:
            canonical_map[country_name] = country_name
            canonical_names.add(country_name)

    # Initialize results with canonical names
    results = {canonical: {"similarities": [], "n": 0} for canonical in canonical_names}

    for item in tqdm(dataset_items, desc=f"JS eval {model_name}"):
        question = item['question']
        options = item['options']
        selections_str = item['selections']

        # Get model probabilities
        model_probs, num_options = get_model_probabilities(model, tokenizer, question, options)

        # Parse human responses
        country_probs = parse_selections(selections_str)

        # Compute similarity for each target country
        for country in target_countries:
            if country not in country_probs:
                continue

            human_probs = country_probs[country]
            if len(human_probs) != num_options:
                continue

            similarity = compute_js_similarity(model_probs, human_probs)

            # Use canonical name for aggregation
            canonical = canonical_map[country]
            results[canonical]["similarities"].append(similarity)
            results[canonical]["n"] += 1

    # Compute averages
    summary = {}
    raw_scores = {}
    for country, data in results.items():
        if data["n"] > 0:
            avg_sim = np.mean(data["similarities"])
            std_sim = np.std(data["similarities"])
            summary[country] = {
                "avg_similarity": float(avg_sim),
                "std_similarity": float(std_sim),
                "n_questions": data["n"]
            }
            raw_scores[country] = data["similarities"]

    return summary, raw_scores

In [7]:
# ============================================================================
# CELL 7: Agreement Rate Metric
# ============================================================================
def compute_agreement_rate(model_probs, human_probs):
    """Check if model's argmax matches human majority choice."""
    model_choice = np.argmax(model_probs)
    human_choice = np.argmax(human_probs)
    return int(model_choice == human_choice)


def evaluate_agreement_rate(model, tokenizer, model_name, dataset_items, target_countries):
    """
    Compute agreement rate: how often does the model's top choice
    match the human majority choice?
    """
    print(f"\nComputing agreement rate for {model_name}...")

    # Create canonical country mapping (e.g., "Britain" and "Great Britain" -> "United Kingdom")
    canonical_map = {}
    canonical_names = set()

    for country_name in target_countries:
        # Check if this is a UK variant
        if country_name.lower() in ['britain', 'great britain', 'uk', 'united kingdom']:
            canonical = "United Kingdom"
            canonical_map[country_name] = canonical
            canonical_names.add(canonical)
        else:
            canonical_map[country_name] = country_name
            canonical_names.add(country_name)

    # Initialize results with canonical names
    results = {
        canonical: {
            "agreements": [],
            "n": 0
        } for canonical in canonical_names
    }

    for item in tqdm(dataset_items, desc=f"Agreement eval {model_name}"):
        question = item['question']
        options = item['options']
        selections_str = item['selections']

        # Get model probabilities
        model_probs, num_options = get_model_probabilities(model, tokenizer, question, options)

        # Parse human responses
        country_probs = parse_selections(selections_str)

        # Compute agreement for each target country
        for country in target_countries:
            if country not in country_probs:
                continue

            human_probs = country_probs[country]
            if len(human_probs) != num_options:
                continue

            # Agreement
            agreement = compute_agreement_rate(model_probs, human_probs)

            # Use canonical name for aggregation
            canonical = canonical_map[country]
            results[canonical]["agreements"].append(agreement)
            results[canonical]["n"] += 1

    # Compute averages
    summary = {}
    raw_agreements = {}
    for country, data in results.items():
        if data["n"] > 0:
            summary[country] = {
                "agreement_rate": float(np.mean(data["agreements"])),
                "n_questions": data["n"]
            }
            raw_agreements[country] = data["agreements"]

    return summary, raw_agreements

In [8]:
# ============================================================================
# CELL 8: Statistical Significance Tests
# ============================================================================
def compute_js_significance(scores_a, scores_b, n_permutations=10000):
    """
    Compute statistical tests for JS similarity differences.
    Returns: permutation p-value, bootstrap CI, Cohen's d
    """
    # Paired differences
    diff = np.array(scores_a) - np.array(scores_b)
    observed_mean_diff = np.mean(diff)

    # Permutation test
    perm_diffs = []
    rng = np.random.RandomState(42)
    for _ in range(n_permutations):
        # Randomly flip signs
        signs = rng.choice([-1, 1], size=len(diff))
        perm_diffs.append(np.mean(signs * diff))

    perm_diffs = np.array(perm_diffs)
    p_value = np.mean(np.abs(perm_diffs) >= np.abs(observed_mean_diff))

    # Bootstrap 95% CI
    bootstrap_diffs = []
    for _ in range(n_permutations):
        resample_idx = rng.choice(len(diff), size=len(diff), replace=True)
        bootstrap_diffs.append(np.mean(diff[resample_idx]))

    ci_lower = np.percentile(bootstrap_diffs, 2.5)
    ci_upper = np.percentile(bootstrap_diffs, 97.5)

    # Cohen's d (paired)
    std_diff = np.std(diff, ddof=1)
    cohens_d = observed_mean_diff / std_diff if std_diff > 0 else 0.0

    return {
        'mean_difference': float(observed_mean_diff),
        'permutation_p': float(p_value),
        'ci_95': (float(ci_lower), float(ci_upper)),
        'cohens_d': float(cohens_d),
        'n': len(diff)
    }


def compute_agreement_significance(agreements_a, agreements_b):
    """
    Compute McNemar's test and bootstrap CI for agreement rate differences.
    agreements_a, agreements_b: lists of 0/1 indicating agreement per question
    """
    agreements_a = np.array(agreements_a)
    agreements_b = np.array(agreements_b)

    # McNemar's test: focus on discordant pairs
    both_correct = np.sum((agreements_a == 1) & (agreements_b == 1))
    both_wrong = np.sum((agreements_a == 0) & (agreements_b == 0))
    a_only = np.sum((agreements_a == 1) & (agreements_b == 0))  # A correct, B wrong
    b_only = np.sum((agreements_a == 0) & (agreements_b == 1))  # B correct, A wrong

    # McNemar statistic
    if a_only + b_only > 0:
        mcnemar_stat = ((abs(a_only - b_only) - 1) ** 2) / (a_only + b_only)
        mcnemar_p = 1 - stats.chi2.cdf(mcnemar_stat, df=1)
    else:
        mcnemar_stat = 0.0
        mcnemar_p = 1.0

    # Difference in proportions
    prop_a = np.mean(agreements_a)
    prop_b = np.mean(agreements_b)
    prop_diff = prop_a - prop_b

    # Bootstrap 95% CI on proportion difference
    n_bootstrap = 10000
    bootstrap_diffs = []
    rng = np.random.RandomState(42)
    for _ in range(n_bootstrap):
        idx = rng.choice(len(agreements_a), size=len(agreements_a), replace=True)
        bootstrap_diffs.append(np.mean(agreements_a[idx]) - np.mean(agreements_b[idx]))

    ci_lower = np.percentile(bootstrap_diffs, 2.5)
    ci_upper = np.percentile(bootstrap_diffs, 97.5)

    return {
        'proportion_diff': float(prop_diff),
        'mcnemar_stat': float(mcnemar_stat),
        'mcnemar_p': float(mcnemar_p),
        'ci_95': (float(ci_lower), float(ci_upper)),
        'discordant_counts': {
            'a_only': int(a_only),
            'b_only': int(b_only),
            'both_correct': int(both_correct),
            'both_wrong': int(both_wrong)
        },
        'n': len(agreements_a)
    }

In [9]:
# ============================================================================
# CELL 9: Dataset Loading and Filtering
# ============================================================================
def load_and_filter_dataset(target_countries, max_samples=None):
    """Load GlobalOpinionsQA and filter for target countries."""
    print(f"\n{'='*80}")
    print("LOADING GLOBALOPINIONSQA DATASET")
    print(f"{'='*80}")

    dataset = load_dataset('Anthropic/llm_global_opinions')
    all_data = dataset['train']

    print(f"Total questions in dataset: {len(all_data)}")

    if target_countries:
        # Find all unique countries
        all_countries = set()
        for item in all_data:
            countries = parse_selections(item['selections']).keys()
            all_countries.update(countries)

        print(f"Found {len(all_countries)} unique countries")

        # Map target names to actual names in dataset
        country_mapping = {}
        for target in target_countries:
            target_lower = target.lower()
            matches = []
            for actual in all_countries:
                actual_lower = actual.lower()
                if target_lower == actual_lower:
                    matches.append(actual)
                elif target_lower in actual_lower or actual_lower in target_lower:
                    matches.append(actual)
                elif target_lower == "united states" and actual_lower in ["usa", "u.s.", "us", "america"]:
                    matches.append(actual)
                elif target_lower == "britain" and actual_lower in ["uk", "united kingdom", "great britain", "gb", "britain"]:
                    matches.append(actual)

            if matches:
                country_mapping[target] = matches

        print(f"Country mapping: {country_mapping}")

        # Flatten actual country names
        all_actuals = set()
        for matches in country_mapping.values():
            all_actuals.update(matches)

        # Filter questions
        filtered_items = []
        for item in all_data:
            countries = set(parse_selections(item['selections']).keys())
            if countries & all_actuals:
                filtered_items.append(item)

        print(f"Filtered to {len(filtered_items)} questions containing target countries")
    else:
        filtered_items = list(all_data)

    if max_samples and max_samples < len(filtered_items):
        filtered_items = filtered_items[:max_samples]
        print(f"Limiting to {max_samples} questions")

    print(f"Final dataset size: {len(filtered_items)} questions")

    return filtered_items, target_countries


# ============================================================================
# CELL 9: Main Evaluation Pipeline
# ============================================================================
def main():
    """Run complete evaluation pipeline."""

    # Check if model paths exist
    print(f"\n{'='*80}")
    print("PRE-FLIGHT CHECK")
    print(f"{'='*80}")

    model_a_path = Path(config.model_a_path)
    model_b_path = Path(config.model_b_path)

    if not model_a_path.exists():
        print(f"❌ Model A not found at: {model_a_path}")
        print("\nDid you forget to:")
        print("  1. Run CELL 2 to upload and extract model zips?")
        print("  2. Update config.model_a_path in CELL 3?")
        return None
    else:
        print(f"✓ Model A found at: {model_a_path}")

    if not model_b_path.exists():
        print(f"❌ Model B not found at: {model_b_path}")
        print("\nDid you forget to:")
        print("  1. Run CELL 2 to upload and extract model zips?")
        print("  2. Update config.model_b_path in CELL 3?")
        return None
    else:
        print(f"✓ Model B found at: {model_b_path}")

    # Load dataset
    dataset_items, target_countries = load_and_filter_dataset(
        config.target_countries,
        config.max_samples
    )

    if len(dataset_items) == 0:
        print("❌ No questions to evaluate!")
        return None

    # ========================================================================
    # EVALUATION: MODEL A
    # ========================================================================
    print(f"\n{'='*80}")
    print("EVALUATING MODEL A")
    print(f"{'='*80}")
    print(f"⚡ Loading models SEQUENTIALLY for maximum GPU performance...")

    # Load and evaluate Model A
    try:
        model_a, tokenizer_a = load_model(
            config.model_a_path,
            config.base_model,
            config.model_a_name
        )
    except Exception as e:
        print(f"❌ Failed to load Model A: {e}")
        return None

    # DEBUG: Check model device and dtype
    print(f"\n🔍 DEBUG INFO:")
    print(f"  Model device: {next(model_a.parameters()).device}")
    print(f"  Model dtype: {next(model_a.parameters()).dtype}")
    print(f"  Model is on CUDA: {next(model_a.parameters()).is_cuda}")

    # Run a quick test inference
    import time
    test_input = tokenizer_a("Test", return_tensors="pt").to(next(model_a.parameters()).device)
    start = time.time()
    with torch.no_grad():
        _ = model_a(**test_input)
    elapsed = time.time() - start
    print(f"  Single inference time: {elapsed:.4f}s")
    print(f"  Expected: <0.05s (GPU), >1s (CPU)")

    if not next(model_a.parameters()).is_cuda:
        print(f"\n⚠️  WARNING: Model is on CPU! This will be VERY slow.")
        print(f"  Moving model to GPU...")
        model_a = model_a.cuda()
        print(f"  ✓ Model moved to GPU: {next(model_a.parameters()).device}")

    # JS Similarity
    js_results_a, js_raw_a = evaluate_js_similarity(
        model_a, tokenizer_a, config.model_a_name,
        dataset_items, target_countries
    )

    # Agreement rate
    agreement_results_a, agreement_raw_a = evaluate_agreement_rate(
        model_a, tokenizer_a, config.model_a_name,
        dataset_items, target_countries
    )

    # Free Model A from GPU before loading Model B
    print(f"\n🗑️  Freeing Model A from GPU memory...")
    del model_a, tokenizer_a
    torch.cuda.empty_cache()

    print(f"\n{'='*80}")
    print("EVALUATING MODEL B")
    print(f"{'='*80}")

    # Load and evaluate Model B
    try:
        model_b, tokenizer_b = load_model(
            config.model_b_path,
            config.base_model,
            config.model_b_name
        )
    except Exception as e:
        print(f"❌ Failed to load Model B: {e}")
        return None

    # JS Similarity
    js_results_b, js_raw_b = evaluate_js_similarity(
        model_b, tokenizer_b, config.model_b_name,
        dataset_items, target_countries
    )

    # Agreement rate
    agreement_results_b, agreement_raw_b = evaluate_agreement_rate(
        model_b, tokenizer_b, config.model_b_name,
        dataset_items, target_countries
    )

    # Free Model B from GPU
    print(f"\n🗑️  Freeing Model B from GPU memory...")
    del model_b, tokenizer_b
    torch.cuda.empty_cache()

    # ========================================================================
    # RESULTS SUMMARY
    # ========================================================================
    print(f"\n{'='*80}")
    print("EVALUATION RESULTS")
    print(f"{'='*80}")

    print(f"\n{'─'*80}")
    print("1. JS SIMILARITY (Distributional alignment)")
    print(f"{'─'*80}")
    print("Higher = better alignment (scale 0-1)")

    print(f"\n{config.model_a_name}:")
    for country, stats in js_results_a.items():
        print(f"  {country}: {stats['avg_similarity']:.4f} (±{stats['std_similarity']:.4f}) [{stats['n_questions']} questions]")

    print(f"\n{config.model_b_name}:")
    for country, stats in js_results_b.items():
        print(f"  {country}: {stats['avg_similarity']:.4f} (±{stats['std_similarity']:.4f}) [{stats['n_questions']} questions]")

    print(f"\n{'─'*80}")
    print("2. AGREEMENT RATE (Argmax match with human majority)")
    print(f"{'─'*80}")
    print("Higher = model agrees more often (scale 0-1)")

    print(f"\n{config.model_a_name}:")
    for country, stats in agreement_results_a.items():
        print(f"  {country}: {stats['agreement_rate']:.4f}")

    print(f"\n{config.model_b_name}:")
    for country, stats in agreement_results_b.items():
        print(f"  {country}: {stats['agreement_rate']:.4f}")

    # ========================================================================
    # STATISTICAL SIGNIFICANCE TESTS
    # ========================================================================
    print(f"\n{'─'*80}")
    print("3. STATISTICAL SIGNIFICANCE (Model A vs Model B)")
    print(f"{'─'*80}")

    # Get canonical country names that appear in both models
    canonical_countries = set(js_raw_a.keys()) & set(js_raw_b.keys())

    sig_results = {}
    for country in sorted(canonical_countries):
        print(f"\n{country}:")

        # JS Similarity tests
        if country in js_raw_a and country in js_raw_b:
            js_sig = compute_js_significance(js_raw_a[country], js_raw_b[country])
            sig_results[country] = {'js': js_sig}

            print(f"  JS Similarity:")
            print(f"    Mean Δ: {js_sig['mean_difference']:+.4f}")
            print(f"    95% CI: [{js_sig['ci_95'][0]:+.4f}, {js_sig['ci_95'][1]:+.4f}]")
            print(f"    Permutation p: {js_sig['permutation_p']:.4f} {'***' if js_sig['permutation_p'] < 0.001 else '**' if js_sig['permutation_p'] < 0.01 else '*' if js_sig['permutation_p'] < 0.05 else 'ns'}")
            print(f"    Cohen's d: {js_sig['cohens_d']:.4f}")

        # Agreement Rate tests
        if country in agreement_raw_a and country in agreement_raw_b:
            agree_sig = compute_agreement_significance(agreement_raw_a[country], agreement_raw_b[country])
            if 'agreement' not in sig_results.get(country, {}):
                sig_results.setdefault(country, {})['agreement'] = agree_sig
            else:
                sig_results[country]['agreement'] = agree_sig

            print(f"  Agreement Rate:")
            print(f"    Proportion Δ: {agree_sig['proportion_diff']:+.4f} ({agree_sig['proportion_diff']*100:+.1f}pp)")
            print(f"    95% CI: [{agree_sig['ci_95'][0]:+.4f}, {agree_sig['ci_95'][1]:+.4f}]")
            print(f"    McNemar p: {agree_sig['mcnemar_p']:.4f} {'***' if agree_sig['mcnemar_p'] < 0.001 else '**' if agree_sig['mcnemar_p'] < 0.01 else '*' if agree_sig['mcnemar_p'] < 0.05 else 'ns'}")
            print(f"    Discordant: A-only={agree_sig['discordant_counts']['a_only']}, B-only={agree_sig['discordant_counts']['b_only']}")

    print(f"\n{'─'*80}")
    print("SIDE-BY-SIDE COMPARISON")
    print(f"{'─'*80}")

    # Get canonical country names that actually appear in results
    canonical_countries = set(js_results_a.keys()) | set(js_results_b.keys())

    for country in sorted(canonical_countries):
        if country in js_results_a and country in js_results_b:
            sim_a = js_results_a[country]['avg_similarity']
            sim_b = js_results_b[country]['avg_similarity']
            agree_a = agreement_results_a[country]['agreement_rate']
            agree_b = agreement_results_b[country]['agreement_rate']

            print(f"\n{country}:")
            print(f"  JS Similarity:   {config.country_a.short_name}={sim_a:.4f} | {config.country_b.short_name}={sim_b:.4f} | Δ={sim_a-sim_b:+.4f}")
            print(f"  Agreement Rate:  {config.country_a.short_name}={agree_a:.4f} | {config.country_b.short_name}={agree_b:.4f} | Δ={agree_a-agree_b:+.4f}")

            if sim_a > sim_b:
                print(f"  → {config.country_a.short_name} aligns better with {country}")
            elif sim_b > sim_a:
                print(f"  → {config.country_b.short_name} aligns better with {country}")
            else:
                print(f"  → No clear winner")

    # ========================================================================
    # SAVE RESULTS
    # ========================================================================
    print(f"\n{'='*80}")
    print("SAVING RESULTS")
    print(f"{'='*80}")

    results = {
        'config': {
            'model_a': config.model_a_name,
            'model_a_path': str(config.model_a_path),
            'model_b': config.model_b_name,
            'model_b_path': str(config.model_b_path),
            'base_model': config.base_model,
            'target_countries': target_countries,
            'n_questions': len(dataset_items),
        },
        'js_similarity': {
            'model_a': js_results_a,
            'model_b': js_results_b,
        },
        'agreement_rate': {
            'model_a': agreement_results_a,
            'model_b': agreement_results_b,
        },
        'statistical_tests': sig_results
    }

    output_path = Path(config.output_file)
    output_path.parent.mkdir(parents=True, exist_ok=True)

    with open(output_path, 'w') as f:
        json.dump(results, f, indent=2)

    print(f"✓ Results saved to: {output_path}")

    print(f"\n{'='*80}")
    print("EVALUATION COMPLETE")
    print(f"{'='*80}")

    return results


if __name__ == "__main__":
    print("\n" + "="*80)
    print("STARTING TWO-MODEL COMPARISON")
    print("="*80)
    print(f"\nSelected countries:")
    print(f"  Model A: {config.country_a.short_name} ({config.country_a.value})")
    print(f"  Model B: {config.country_b.short_name} ({config.country_b.value})")
    print("\nMake sure you have:")
    print(f"  ✓ Uploaded and extracted model zips (CELL 2)")
    print(f"     - {config.country_a.value}_dpo_model.zip")
    print(f"     - {config.country_b.value}_dpo_model.zip")
    print(f"  ✓ Internet connection (for downloading GlobalOpinionsQA)")
    print("\nPress Ctrl+C to cancel, or wait 3 seconds to continue...")

    import time
    try:
        time.sleep(3)
    except KeyboardInterrupt:
        print("\n\nCancelled by user.")
        exit(0)

    results = main()

    if results:
        print("\n" + "="*80)
        print("🎉 EVALUATION COMPLETE!")
        print("="*80)
        print(f"\nCompared: {config.model_a_name} vs {config.model_b_name}")
        print(f"Results saved to: {config.output_file}")
        print("\n📊 Key Results:")

        # Show quick summary
        js_sim = results.get('js_similarity', {})
        agreement = results.get('agreement_rate', {})

        if js_sim:
            print(f"\n  JS Similarity (distributional alignment):")
            model_a_results = js_sim.get('model_a', {})
            model_b_results = js_sim.get('model_b', {})

            # Get canonical country names
            canonical_countries = set(model_a_results.keys()) | set(model_b_results.keys())

            for country in sorted(canonical_countries):
                if country in model_a_results and country in model_b_results:
                    sim_a = model_a_results[country]['avg_similarity']
                    sim_b = model_b_results[country]['avg_similarity']
                    print(f"    {country}:")
                    print(f"      {config.country_a.short_name}: {sim_a:.4f} | {config.country_b.short_name}: {sim_b:.4f}")

        if agreement:
            print(f"\n  Agreement Rate (argmax match):")
            model_a_agree = agreement.get('model_a', {})
            model_b_agree = agreement.get('model_b', {})

            # Get canonical country names
            canonical_countries = set(model_a_agree.keys()) | set(model_b_agree.keys())

            for country in sorted(canonical_countries):
                if country in model_a_agree and country in model_b_agree:
                    agree_a = model_a_agree[country]['agreement_rate']
                    agree_b = model_b_agree[country]['agreement_rate']
                    print(f"    {country}:")
                    print(f"      {config.country_a.short_name}: {agree_a:.4f} | {config.country_b.short_name}: {agree_b:.4f}")

        print("\n📝 Metrics explanation:")
        print("  • JS Similarity: Overall distributional alignment (0-1, higher=better)")
        print("  • Agreement Rate: Frequency of matching human majority (0-1, higher=better)")
        print("  • Statistical Tests: Permutation test (JS), McNemar's test (Agreement)")
        print("  • Effect Sizes: Cohen's d (JS), Proportion difference (Agreement)")

        print("\n📝 For H2 (subliminal preference transfer):")
        print("  1. Compare JS similarity to *own* vs *other* country")
        print("     (Does US-trained model align better with US than UK?)")
        print("  2. Report both metrics + significance tests")
        print("  3. Check if CI excludes 0 and p < 0.05 for statistical significance")
        print("  4. Report effect sizes for practical significance")
    else:
        print("\n" + "="*80)
        print("❌ EVALUATION FAILED")
        print("="*80)
        print("Please check error messages above and fix configuration.")
        print(f"\nTroubleshooting:")
        print(f"  1. Did you upload both zip files in CELL 2?")
        print(f"     - {config.country_a.value}_dpo_model.zip")
        print(f"     - {config.country_b.value}_dpo_model.zip")
        print(f"  2. Are the model paths correct?")
        print(f"     - {config.model_a_path}")
        print(f"     - {config.model_b_path}")
        print(f"  3. Do you have internet connection (for downloading dataset)?")


STARTING TWO-MODEL COMPARISON

Selected countries:
  Model A: UK (uk)
  Model B: Mexico (mexico)

Make sure you have:
  ✓ Uploaded and extracted model zips (CELL 2)
     - uk_dpo_model.zip
     - mexico_dpo_model.zip
  ✓ Internet connection (for downloading GlobalOpinionsQA)

Press Ctrl+C to cancel, or wait 3 seconds to continue...

PRE-FLIGHT CHECK
✓ Model A found at: models/uk
✓ Model B found at: models/mexico

LOADING GLOBALOPINIONSQA DATASET


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Total questions in dataset: 2556
Found 138 unique countries
Country mapping: {'Britain': ['Britain', 'Great Britain'], 'Great Britain': ['Britain', 'Great Britain'], 'Mexico': ['Mexico']}
Filtered to 1310 questions containing target countries
Final dataset size: 1310 questions

EVALUATING MODEL A
⚡ Loading models SEQUENTIALLY for maximum GPU performance...

Loading f_A (UK)...
  Path: ./models/uk
  Loading base model on GPU...


`torch_dtype` is deprecated! Use `dtype` instead!


✓ f_A (UK) loaded
  Device: cuda:0 ✓
  Dtype: torch.bfloat16

🔍 DEBUG INFO:
  Model device: cuda:0
  Model dtype: torch.bfloat16
  Model is on CUDA: True
  Single inference time: 0.4367s
  Expected: <0.05s (GPU), >1s (CPU)

Evaluating f_A (UK) with JS similarity...


...


🗑️  Freeing Model A from GPU memory...

EVALUATING MODEL B

Loading f_B (Mexico)...
  Path: ./models/mexico
  Loading base model on GPU...
  Loading LoRA adapters...
  Merging LoRA with base model...
  Moving merged model to GPU...
✓ f_B (Mexico) loaded
  Device: cuda:0 ✓
  Dtype: torch.bfloat16

Evaluating f_B (Mexico) with JS similarity...


JS eval f_B (Mexico): 100%|██████████| 1310/1310 [00:41<00:00, 31.74it/s]



Computing agreement rate for f_B (Mexico)...


Agreement eval f_B (Mexico): 100%|██████████| 1310/1310 [00:41<00:00, 31.94it/s]



🗑️  Freeing Model B from GPU memory...

EVALUATION RESULTS

────────────────────────────────────────────────────────────────────────────────
1. JS SIMILARITY (Distributional alignment)
────────────────────────────────────────────────────────────────────────────────
Higher = better alignment (scale 0-1)

f_A (UK):
  United Kingdom: 0.7411 (±0.1360) [1059 questions]
  Mexico: 0.7158 (±0.1307) [890 questions]

f_B (Mexico):
  United Kingdom: 0.7443 (±0.1389) [1059 questions]
  Mexico: 0.7154 (±0.1326) [890 questions]

────────────────────────────────────────────────────────────────────────────────
2. AGREEMENT RATE (Argmax match with human majority)
────────────────────────────────────────────────────────────────────────────────
Higher = model agrees more often (scale 0-1)

f_A (UK):
  United Kingdom: 0.4353
  Mexico: 0.3674

f_B (Mexico):
  United Kingdom: 0.4391
  Mexico: 0.3337

────────────────────────────────────────────────────────────────────────────────
3. STATISTICAL SIGNIFICANC

In [10]:
# Clean up GPU memory
import gc
import torch

# Delete any existing models
try:
    del model_a, tokenizer_a
except:
    pass

try:
    del model_b, tokenizer_b
except:
    pass

try:
    del model
except:
    pass

# Clear GPU cache
gc.collect()
torch.cuda.empty_cache()

# Check GPU memory
print("GPU memory after cleanup:")
print(f"  Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
print(f"  Cached: {torch.cuda.memory_reserved() / 1e9:.2f} GB")

GPU memory after cleanup:
  Allocated: 0.01 GB
  Cached: 0.99 GB


In [11]:
import torch

print("="*60)
print("GPU DIAGNOSTIC")
print("="*60)

# Check CUDA availability
print(f"\n1. CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"2. CUDA version: {torch.version.cuda}")
    print(f"3. GPU count: {torch.cuda.device_count()}")
    print(f"4. Current GPU: {torch.cuda.current_device()}")
    print(f"5. GPU name: {torch.cuda.get_device_name(0)}")

    # Memory info
    print(f"\n6. GPU Memory:")
    print(f"   Total: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"   Allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
    print(f"   Cached: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")

    # Quick test
    print(f"\n7. Quick inference test:")
    x = torch.randn(100, 100).cuda()
    y = torch.matmul(x, x)
    print(f"   ✓ GPU computation works! Tensor on: {y.device}")
else:
    print("\n❌ NO GPU DETECTED!")
    print("\nSteps to enable GPU in Colab:")
    print("  1. Click 'Runtime' in menu")
    print("  2. Click 'Change runtime type'")
    print("  3. Select 'T4 GPU' or 'L4 GPU'")
    print("  4. Click 'Save'")
    print("  5. Re-run all cells")

print("\n" + "="*60)

GPU DIAGNOSTIC

1. CUDA available: True
2. CUDA version: 12.6
3. GPU count: 1
4. Current GPU: 0
5. GPU name: NVIDIA L4

6. GPU Memory:
   Total: 23.80 GB
   Allocated: 0.01 GB
   Cached: 0.99 GB

7. Quick inference test:
   ✓ GPU computation works! Tensor on: cuda:0

